# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** In correspondence with the reccommendation I used a pretrained CNN, resnet 50. Such that I know that the CNN will give a good classification output. Then the output of the CNN is fed into a linear layers for embedding en is feed into an LSTM which has only 1 hidden layer. The reason for 1 hidden layer is that it seemed to give ok results, similar to the discussed paper (Show and Tell: A Neural Image Caption Generator - Oriol Vinyals et al).
The embed size and hidden size was based on the foregoing notebook, I tried a much smaller embed size (10) but training was not good, likely due to the dimensionality reduction important information was lost.
The batch size is chosen as a power of 2 for efficient computation and is set to 256 such that the training process is a bit faster and the learning is a bit smoother.
The vocab threshold remained on 5 based on the Counter in the foregoing notebook. 





### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** I left it at is, since I like the random cropping and flipping to make the CNN more robust. Normalize is good for centering the data closer to 0, which the training process likes, I have no intuitive feeling how to change the normalization parameters.

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** I used a pretrained CNN, resnet 50. Therefore I only want to train the RNN and the embedding of the encoder. Circumvents additional training time and with the limited training I will do, will likely make a better overall model. 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** Selected is for the ADAM optimizer that adaptively adjust the learning rate. 

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 256           # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256         # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.87s)
creating index...


  0%|          | 784/414113 [00:00<01:51, 3716.16it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:34<00:00, 4370.12it/s]


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [2]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder2-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder2-%d.pkl' % epoch))

# Close the training log file.
f.close()

forward pass
Epoch [1/3], Step [1/1618], Loss: 9.0915, Perplexity: 8879.6418forward pass
Epoch [1/3], Step [2/1618], Loss: 8.9583, Perplexity: 7772.2271forward pass
Epoch [1/3], Step [3/1618], Loss: 8.7887, Perplexity: 6559.5433forward pass
Epoch [1/3], Step [4/1618], Loss: 8.5705, Perplexity: 5273.8585forward pass
Epoch [1/3], Step [5/1618], Loss: 8.1099, Perplexity: 3327.3801forward pass
Epoch [1/3], Step [6/1618], Loss: 7.4879, Perplexity: 1786.3281forward pass
Epoch [1/3], Step [7/1618], Loss: 6.6719, Perplexity: 789.8598forward pass
Epoch [1/3], Step [8/1618], Loss: 5.9351, Perplexity: 378.0713forward pass
Epoch [1/3], Step [9/1618], Loss: 5.4098, Perplexity: 223.5763forward pass
Epoch [1/3], Step [10/1618], Loss: 5.3531, Perplexity: 211.2619forward pass
Epoch [1/3], Step [11/1618], Loss: 5.0169, Perplexity: 150.9373forward pass
Epoch [1/3], Step [12/1618], Loss: 4.7738, Perplexity: 118.3729forward pass
Epoch [1/3], Step [13/1618], Loss: 4.6594, Perplexity: 105.5747forward pass
Ep

Epoch [1/3], Step [216/1618], Loss: 3.1854, Perplexity: 24.1774forward pass
Epoch [1/3], Step [217/1618], Loss: 3.4736, Perplexity: 32.2531forward pass
Epoch [1/3], Step [218/1618], Loss: 3.3137, Perplexity: 27.4871forward pass
Epoch [1/3], Step [219/1618], Loss: 3.4061, Perplexity: 30.1461forward pass
Epoch [1/3], Step [220/1618], Loss: 3.7975, Perplexity: 44.5892forward pass
Epoch [1/3], Step [221/1618], Loss: 3.3379, Perplexity: 28.1598forward pass
Epoch [1/3], Step [222/1618], Loss: 3.4203, Perplexity: 30.5776forward pass
Epoch [1/3], Step [223/1618], Loss: 3.4651, Perplexity: 31.9804forward pass
Epoch [1/3], Step [224/1618], Loss: 3.2256, Perplexity: 25.1697forward pass
Epoch [1/3], Step [225/1618], Loss: 3.3450, Perplexity: 28.3609forward pass
Epoch [1/3], Step [226/1618], Loss: 3.3320, Perplexity: 27.9936forward pass
Epoch [1/3], Step [227/1618], Loss: 3.4132, Perplexity: 30.3609forward pass
Epoch [1/3], Step [228/1618], Loss: 3.3355, Perplexity: 28.0934forward pass
Epoch [1/3],

Epoch [1/3], Step [430/1618], Loss: 3.1727, Perplexity: 23.8726forward pass
Epoch [1/3], Step [431/1618], Loss: 2.9610, Perplexity: 19.3169forward pass
Epoch [1/3], Step [432/1618], Loss: 2.9112, Perplexity: 18.3791forward pass
Epoch [1/3], Step [433/1618], Loss: 3.0292, Perplexity: 20.6802forward pass
Epoch [1/3], Step [434/1618], Loss: 3.2177, Perplexity: 24.9704forward pass
Epoch [1/3], Step [435/1618], Loss: 3.1659, Perplexity: 23.7104forward pass
Epoch [1/3], Step [436/1618], Loss: 2.9895, Perplexity: 19.8762forward pass
Epoch [1/3], Step [437/1618], Loss: 2.9762, Perplexity: 19.6137forward pass
Epoch [1/3], Step [438/1618], Loss: 3.0213, Perplexity: 20.5186forward pass
Epoch [1/3], Step [439/1618], Loss: 3.4751, Perplexity: 32.3016forward pass
Epoch [1/3], Step [440/1618], Loss: 3.2029, Perplexity: 24.6044forward pass
Epoch [1/3], Step [441/1618], Loss: 3.4526, Perplexity: 31.5809forward pass
Epoch [1/3], Step [442/1618], Loss: 3.4337, Perplexity: 30.9919forward pass
Epoch [1/3],

Epoch [1/3], Step [644/1618], Loss: 2.8045, Perplexity: 16.5193forward pass
Epoch [1/3], Step [645/1618], Loss: 2.7389, Perplexity: 15.4697forward pass
Epoch [1/3], Step [646/1618], Loss: 2.7298, Perplexity: 15.3292forward pass
Epoch [1/3], Step [647/1618], Loss: 2.8170, Perplexity: 16.7265forward pass
Epoch [1/3], Step [648/1618], Loss: 2.6776, Perplexity: 14.5503forward pass
Epoch [1/3], Step [649/1618], Loss: 3.5072, Perplexity: 33.3537forward pass
Epoch [1/3], Step [650/1618], Loss: 3.4212, Perplexity: 30.6063forward pass
Epoch [1/3], Step [651/1618], Loss: 2.7191, Perplexity: 15.1669forward pass
Epoch [1/3], Step [652/1618], Loss: 2.9077, Perplexity: 18.3150forward pass
Epoch [1/3], Step [653/1618], Loss: 2.7170, Perplexity: 15.1348forward pass
Epoch [1/3], Step [654/1618], Loss: 2.7901, Perplexity: 16.2826forward pass
Epoch [1/3], Step [655/1618], Loss: 2.9036, Perplexity: 18.2406forward pass
Epoch [1/3], Step [656/1618], Loss: 2.8416, Perplexity: 17.1427forward pass
Epoch [1/3],

Epoch [1/3], Step [858/1618], Loss: 2.7418, Perplexity: 15.5148forward pass
Epoch [1/3], Step [859/1618], Loss: 2.5422, Perplexity: 12.7075forward pass
Epoch [1/3], Step [860/1618], Loss: 2.6749, Perplexity: 14.5106forward pass
Epoch [1/3], Step [861/1618], Loss: 2.6464, Perplexity: 14.1038forward pass
Epoch [1/3], Step [862/1618], Loss: 2.5365, Perplexity: 12.6349forward pass
Epoch [1/3], Step [863/1618], Loss: 3.7845, Perplexity: 44.0154forward pass
Epoch [1/3], Step [864/1618], Loss: 2.5946, Perplexity: 13.3915forward pass
Epoch [1/3], Step [865/1618], Loss: 2.6422, Perplexity: 14.0440forward pass
Epoch [1/3], Step [866/1618], Loss: 2.5827, Perplexity: 13.2326forward pass
Epoch [1/3], Step [867/1618], Loss: 2.5508, Perplexity: 12.8168forward pass
Epoch [1/3], Step [868/1618], Loss: 2.6223, Perplexity: 13.7680forward pass
Epoch [1/3], Step [869/1618], Loss: 2.4538, Perplexity: 11.6329forward pass
Epoch [1/3], Step [870/1618], Loss: 3.0243, Perplexity: 20.5800forward pass
Epoch [1/3],

...

Epoch [3/3], Step [751/1618], Loss: 2.0411, Perplexity: 7.6992forward pass
Epoch [3/3], Step [752/1618], Loss: 1.9726, Perplexity: 7.1896forward pass
Epoch [3/3], Step [753/1618], Loss: 1.9706, Perplexity: 7.1750forward pass
Epoch [3/3], Step [754/1618], Loss: 2.0481, Perplexity: 7.7530forward pass
Epoch [3/3], Step [755/1618], Loss: 2.0668, Perplexity: 7.8999forward pass
Epoch [3/3], Step [756/1618], Loss: 2.1165, Perplexity: 8.3019forward pass
Epoch [3/3], Step [757/1618], Loss: 2.0328, Perplexity: 7.6354forward pass
Epoch [3/3], Step [758/1618], Loss: 1.9855, Perplexity: 7.2824forward pass
Epoch [3/3], Step [759/1618], Loss: 1.8736, Perplexity: 6.5114forward pass
Epoch [3/3], Step [760/1618], Loss: 2.7581, Perplexity: 15.7697forward pass
Epoch [3/3], Step [761/1618], Loss: 2.3583, Perplexity: 10.5730forward pass
Epoch [3/3], Step [762/1618], Loss: 2.1469, Perplexity: 8.5583forward pass
Epoch [3/3], Step [763/1618], Loss: 2.2525, Perplexity: 9.5117forward pass
Epoch [3/3], Step [764/

Epoch [3/3], Step [969/1618], Loss: 2.0244, Perplexity: 7.5718forward pass
Epoch [3/3], Step [970/1618], Loss: 2.0527, Perplexity: 7.7893forward pass
Epoch [3/3], Step [971/1618], Loss: 2.0237, Perplexity: 7.5660forward pass
Epoch [3/3], Step [972/1618], Loss: 2.2326, Perplexity: 9.3241forward pass
Epoch [3/3], Step [973/1618], Loss: 2.0595, Perplexity: 7.8422forward pass
Epoch [3/3], Step [974/1618], Loss: 2.2639, Perplexity: 9.6202forward pass
Epoch [3/3], Step [975/1618], Loss: 2.1675, Perplexity: 8.7367forward pass
Epoch [3/3], Step [976/1618], Loss: 1.9408, Perplexity: 6.9643forward pass
Epoch [3/3], Step [977/1618], Loss: 2.2637, Perplexity: 9.6182forward pass
Epoch [3/3], Step [978/1618], Loss: 2.0081, Perplexity: 7.4492forward pass
Epoch [3/3], Step [979/1618], Loss: 1.9505, Perplexity: 7.0324forward pass
Epoch [3/3], Step [980/1618], Loss: 2.0097, Perplexity: 7.4611forward pass
Epoch [3/3], Step [981/1618], Loss: 2.0398, Perplexity: 7.6892forward pass
Epoch [3/3], Step [982/16

Epoch [3/3], Step [1184/1618], Loss: 2.0362, Perplexity: 7.6612forward pass
Epoch [3/3], Step [1185/1618], Loss: 2.2007, Perplexity: 9.0317forward pass
Epoch [3/3], Step [1186/1618], Loss: 2.1870, Perplexity: 8.9081forward pass
Epoch [3/3], Step [1187/1618], Loss: 1.9653, Perplexity: 7.1367forward pass
Epoch [3/3], Step [1188/1618], Loss: 2.2340, Perplexity: 9.3371forward pass
Epoch [3/3], Step [1189/1618], Loss: 1.9211, Perplexity: 6.8285forward pass
Epoch [3/3], Step [1190/1618], Loss: 2.0464, Perplexity: 7.7403forward pass
Epoch [3/3], Step [1191/1618], Loss: 2.1972, Perplexity: 8.9994forward pass
Epoch [3/3], Step [1192/1618], Loss: 1.9778, Perplexity: 7.2269forward pass
Epoch [3/3], Step [1193/1618], Loss: 1.9838, Perplexity: 7.2706forward pass
Epoch [3/3], Step [1194/1618], Loss: 1.9491, Perplexity: 7.0226forward pass
Epoch [3/3], Step [1195/1618], Loss: 2.2814, Perplexity: 9.7900forward pass
Epoch [3/3], Step [1196/1618], Loss: 1.8950, Perplexity: 6.6527forward pass
Epoch [3/3],

Epoch [3/3], Step [1398/1618], Loss: 1.9360, Perplexity: 6.9313forward pass
Epoch [3/3], Step [1399/1618], Loss: 1.9716, Perplexity: 7.1822forward pass
Epoch [3/3], Step [1400/1618], Loss: 2.0461, Perplexity: 7.7374
forward pass
Epoch [3/3], Step [1401/1618], Loss: 1.8680, Perplexity: 6.4756forward pass
Epoch [3/3], Step [1402/1618], Loss: 1.9033, Perplexity: 6.7081forward pass
Epoch [3/3], Step [1403/1618], Loss: 1.9791, Perplexity: 7.2361forward pass
Epoch [3/3], Step [1404/1618], Loss: 1.9853, Perplexity: 7.2812forward pass
Epoch [3/3], Step [1405/1618], Loss: 1.9484, Perplexity: 7.0174forward pass
Epoch [3/3], Step [1406/1618], Loss: 2.1240, Perplexity: 8.3648forward pass
Epoch [3/3], Step [1407/1618], Loss: 2.0525, Perplexity: 7.7872forward pass
Epoch [3/3], Step [1408/1618], Loss: 1.9918, Perplexity: 7.3284forward pass
Epoch [3/3], Step [1409/1618], Loss: 2.0759, Perplexity: 7.9714forward pass
Epoch [3/3], Step [1410/1618], Loss: 1.9840, Perplexity: 7.2720forward pass
Epoch [3/3]

Epoch [3/3], Step [1612/1618], Loss: 2.0149, Perplexity: 7.5001forward pass
Epoch [3/3], Step [1613/1618], Loss: 1.9212, Perplexity: 6.8295forward pass
Epoch [3/3], Step [1614/1618], Loss: 1.9224, Perplexity: 6.8371forward pass
Epoch [3/3], Step [1615/1618], Loss: 1.9923, Perplexity: 7.3326forward pass
Epoch [3/3], Step [1616/1618], Loss: 2.0281, Perplexity: 7.5999forward pass
Epoch [3/3], Step [1617/1618], Loss: 2.0175, Perplexity: 7.5192forward pass
Epoch [3/3], Step [1618/1618], Loss: 2.0071, Perplexity: 7.4420

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.